In [28]:
import sys

import json
from pprint import pprint

import pandas as pd
import spotipy

from configFM import CLIENT_ID, CLIENT_SECRET
from configNI import client_token
import requests
import base64

In [11]:
#TO DO - create a function to take in playlist url & 

In [13]:
# Step 1 - Authorization 
url = "https://accounts.spotify.com/api/token"
headers = {}
data = {}

# Encode as Base64
message = f"{CLIENT_ID}:{CLIENT_SECRET}"
messageBytes = message.encode('ascii')
base64Bytes = base64.b64encode(messageBytes)
base64Message = base64Bytes.decode('ascii')


headers['Authorization'] = f"Basic {base64Message}"
data['grant_type'] = "client_credentials"

r = requests.post(url, headers=headers, data=data)

token = r.json()['access_token']

# Step 2 - Use Access Token to call playlist endpoint

playlistIdNI = "37i9dQZF1ELZnSHLfRnHWB?si=fEi2ScH5S4uAEB5jC3n0yw"
playlistUrlNI = f"https://api.spotify.com/v1/playlists/{playlistIdNI}"
headers = {
    "Authorization": "Bearer " + token
}

res = requests.get(url=playlistUrlNI, headers=headers)

#print(json.dumps(res.json(), indent=2))



In [14]:
playlistIdFM = "37i9dQZF1EM7xNAvCtsxMc?si=LZQ4siPRSQC5ViOxA5FSgA"
playlistUrlFM = f"https://api.spotify.com/v1/playlists/{playlistIdFM}"
headers = {
    "Authorization": "Bearer " + token
}

resFM = requests.get(url=playlistUrlFM, headers=headers)

jsonFM = resFM.json()

pjsonFM = json.dumps(resFM.json(), indent=2)
#print(pjsonFM)

In [15]:
jsonFM["tracks"]["items"][0]["track"]["artists"][0]["name"]
jsonFM["tracks"]["items"][0]["track"]["name"]

'Waves'

In [16]:
artist = []
song = []
album = []

for i,t in enumerate (jsonFM["tracks"]["items"]):
    artist.append(t["track"]["artists"][0]["name"])
    album.append(t["track"]["album"]["name"])
    song.append(t["track"]["name"])
    
final = {
    "Name": song,
    "Album":album,
    "Artist":artist
}

In [17]:
playlist_df = pd.DataFrame(final)
playlist_df

,Name,Album,Artist
0,Waves,The Life Of Pablo,Kanye West
1,Lord Knows (feat. Tory Lanez),Dreams Worth More Than Money,Meek Mill
2,GOOBA,GOOBA,6ix9ine
3,Moon Relate,Eternal Atake (Deluxe) - LUV vs. The World 2,Lil Uzi Vert
4,On God,JESUS IS KING,Kanye West
...,...,...,...
95,Runnin,SAVAGE MODE II,21 Savage
96,BILLY,Day69: Graduation Day,6ix9ine
97,All I Want,In A Perfect World (Expanded Edition),Kodaline
98,Sunday Candy,Surf,Donnie Trumpet & The Social Experiment


In [18]:
pip install lyricsgenius

Note: you may need to restart the kernel to use updated packages.


In [19]:
import lyricsgenius
genius = lyricsgenius.Genius(client_token)
artist = genius.search_artist("Pop Smoke", max_songs=3, sort="title")
print(artist.songs)

Searching for songs by Pop Smoke...

Song 1: "100k On a Coupe"
Song 2: "2020*"
Song 3: "44 BullDog"

Reached user-specified song limit (3).
Done. Found 3 songs.
[('100k On a Coupe', 'Pop Smoke'), ('2020*', 'Pop Smoke'), ('44 BullDog', 'Pop Smoke')]


In [20]:
song = genius.search_song("Dior", artist.name)
print(song.lyrics)

Searching for "Dior" by Pop Smoke...
Done.
[Intro]
Traphouse Mob
Huh,  roll another one
Said  I'm never lackin', always pistol packing
With them automatics, we gon' send him to heaven
Wait, wait, wait, wait, ayy, ayy, woo (Aw, shit), huh
Oh,  you feelin' sturdy, huh? (You feelin' sturdy, man)
Shake  somethin'
Shake it, shake it, shake it

[Chorus]
She  like the way that I dance
She like the way that I move
She like the way that I rock
She like the way that I woo
And  she let it clap for a nigga
(She let it clap for a nigga)
And she throw it back for a nigga
(Yeah, she throw it back for a nigga)
Mike Amiri, Mike Amiri
Billie Jean, Billie Jean (Uh)
Christian Dior, Dior
I'm up in all the stores
When it rains, it pours
She like the way I rrr
Mike Amiri, Mike Amiri
Billie Jean, Billie Jean (Uh)
Christian Dior, Dior
I'm up in all the stores
When it rains, it pours
She like the way I rrr

[Verse]
When I walk in the spot, thirty on me
Buy out the club, niggas know that I'm paid
Bitch, I'm a th

In [21]:
genius_artist = playlist_df['Artist'].tolist()
genius_song = playlist_df['Name'].tolist()
#print(genius_artist)
#print(genius_song)

In [22]:
genius_lyrics = []
for x in range(len(genius_artist)):
    try:
        genius = lyricsgenius.Genius(client_token)
        artist = genius.search_artist(genius_artist[x], max_songs = 0)
        song = genius.search_song(genius_song[x], artist.name)
        genius_lyrics.append(song.lyrics)
       
    except:
        song = ' '
        genius_lyrics.append(' ')
        
playlist_df['Lyrics'] = genius_lyrics
     
        

Searching for songs by Kanye West...

Done. Found 0 songs.
Searching for "Waves" by Kanye West...
Done.
Searching for songs by Meek Mill...

Done. Found 0 songs.
Searching for "Lord Knows (feat. Tory Lanez)" by Meek Mill...
Done.
Searching for songs by 6ix9ine...

Done. Found 0 songs.
Searching for "GOOBA" by 6ix9ine...
Done.
Searching for songs by Lil Uzi Vert...

Done. Found 0 songs.
Searching for "Moon Relate" by Lil Uzi Vert...
Done.
Searching for songs by Kanye West...

Done. Found 0 songs.
Searching for "On God" by Kanye West...
Done.
Searching for songs by Pop Smoke...

Done. Found 0 songs.
Searching for "What You Know Bout Love" by Pop Smoke...
Done.
Searching for songs by MEDUZA...

Changing artist name to 'Eddie Meduza'
Done. Found 0 songs.
Searching for "Lose Control" by Eddie Meduza...
Done.
Searching for songs by DJ Khaled...

Done. Found 0 songs.
Searching for "We Takin' Over" by DJ Khaled...
Done.
Searching for songs by Kanye West...

Done. Found 0 songs.
Searching for "

Done.
Searching for songs by Don Omar...

Done. Found 0 songs.
Searching for "Danza Kuduro" by Don Omar...
Done.
Searching for songs by DaBaby...

Done. Found 0 songs.
Searching for "Back To Cali" by DaBaby...
Done.
Searching for songs by Britney Spears...

Done. Found 0 songs.
Searching for "Gimme More" by Britney Spears...
Done.
Searching for songs by Young M.A...

Done. Found 0 songs.
Searching for "OOOUUU" by Young M.A...
Done.
Searching for songs by Ariana Grande...

Done. Found 0 songs.
Searching for "7 rings" by Ariana Grande...
Done.
Searching for songs by Black Eyed Peas...

Done. Found 0 songs.
Searching for "RITMO (Bad Boys For Life)" by Black Eyed Peas...
Done.
Searching for songs by Nicky Jam...

Done. Found 0 songs.
Searching for "Hasta el Amanecer" by Nicky Jam...
Done.
Searching for songs by DJ Khaled...

Done. Found 0 songs.
Searching for "Out Here Grindin'" by DJ Khaled...
Done.
Searching for songs by Imagine Dragons...

Done. Found 0 songs.
Searching for "Radioactive

In [23]:
playlist_df

,Name,Album,Artist,Lyrics
0,Waves,The Life Of Pablo,Kanye West,[Intro]\nTurn it up!\n\n[Verse 1: Kanye West]\...
1,Lord Knows (feat. Tory Lanez),Dreams Worth More Than Money,Meek Mill,[Chorus: Tory Lanez]\nAll I wanted was a new M...
2,GOOBA,GOOBA,6ix9ine,"[Chorus]\nI tell a nigga don't dick ride, don'..."
3,Moon Relate,Eternal Atake (Deluxe) - LUV vs. The World 2,Lil Uzi Vert,"[Intro]\nPain, I done got numb from the pain (..."
4,On God,JESUS IS KING,Kanye West,"[Intro]\nYo, Pi'erre, yo, Pi'erre\n\n[Verse]\n..."
...,...,...,...,...
95,Runnin,SAVAGE MODE II,21 Savage,[Intro: Diana Ross & 21 Savage]\nAm I really h...
96,BILLY,Day69: Graduation Day,6ix9ine,"[Intro: Shotti & 6ix9ine]\nThat's my word, get..."
97,All I Want,In A Perfect World (Expanded Edition),Kodaline,[Verse 1]\nAll I want is nothing more\nTo hear...
98,Sunday Candy,Surf,Donnie Trumpet & The Social Experiment,[Produced by Donnie Trumpet and the Social Exp...


In [53]:
playlist_df['Lyrics'] = genius_lyrics
list_lyrics = []
for x in range(len(genius_lyrics)):
    genius_lyrics[x] = genius_lyrics[x].replace("Chorus", " ")
    genius_lyrics[x] = genius_lyrics[x].replace('\n',' ')
    genius_lyrics[x] = genius_lyrics[x].replace('[',' ')
    genius_lyrics[x] = genius_lyrics[x].replace(']',' ')
    genius_lyrics[x] = genius_lyrics[x].replace('Verse',' ')
    genius_lyrics[x] = genius_lyrics[x].replace(':',' ')
    genius_lyrics[x] = genius_lyrics[x].replace('Bridge',' ')
    genius_lyrics[x] = genius_lyrics[x].replace('Hook',' ')
    genius_lyrics[x] = genius_lyrics[x].replace('Intro',' ')
    list_lyrics.append(genius_lyrics[x])
print(list_lyrics)
    

["   I n t r o     T u r n   i t   u p !       V e r s e   1     K a n y e   W e s t     S t e p   u p   i n   t h i s   b i t c h   l i k e   ( T u r n   i t   u p ! )   I ' m   t h e   o n e   y o u r   b i t c h   l i k e   Y e a h   I ' m   t h e   o n e   y o u r   b i t c h   l i k e   ( T u r n   i t   u p ! )   A n d   I   b e   t a l k i n '   s h i t   l i k e   I   a i n ' t   s c a r e d   t o   l o s e   a   f i s t f i g h t   ( T u r n   i t   u p ! )   A n d   s h e   g r a b b i n '   o n   m y ,   l i k e   S h e   w a n n a   s e e   i f   i t ' l l   f i t   r i g h t   ( T u r n   i t   u p ! )   T h a t ' s   j u s t   t h e   w a v e             C h r i s   B r o w n   &   A k o n     ( Y e a h )   W a v e s   d o n ' t   d i e   L e t   m e   c r a s h   h e r e   f o r   t h e   m o m e n t ,   y e a h   I   d o n ' t   n e e d   t o   o w n   i t   N o   l i e   W a v e s   d o n ' t   d i e ,   b a b y   L e t   m e   c r a s h   h e r e   f o r   a   m o m e

In [42]:
playlist_df['Lyrics']=playlist_df['Lyrics'].replace('\n',' ')
playlist_df['Lyrics']=playlist_df['Lyrics'].replace('[',' ')
playlist_df['Lyrics']=playlist_df['Lyrics'].replace(']',' ')
playlist_df['Lyrics']=playlist_df['Lyrics'].replace('Chorus',' ')
playlist_df['Lyrics']=playlist_df['Lyrics'].replace('Verse',' ')
playlist_df['Lyrics']=playlist_df['Lyrics'].replace(':',' ')

In [46]:
print(playlist_df['Lyrics'][1])

[Chorus: Tory Lanez]
All I wanted was a new Mercedes
Bending off the corner whipping out the lot, I got it
Women love me but the niggas hate it
But how can I lose when I came from the bottom?
Lord knows
Nigga Lord knows, nigga Lord knows, nigga Lord knows
Nigga Lord knows, nigga Lord knows, nigga Lord knows
Nigga Lord knows, nigga Lord knows, nigga Lord knows

[Verse 1: Meek Mill]
Uh, Lord knows I'm filthy rich
All this ice is like 50 bricks
Rap niggas throwing hissy fits
I give my bitch a stack just for a Christmas gift
And my bitch so bad, she on my Christmas list
'Member I prayed, really I wished for this
To get the crib with the maid and with the picket fence
I'm with some niggas that mad, we taking risks for this
I'm talking risky business, flick the wrist
Lord knows that I repent for this
But Lord knows if I get penned for this
I prolly won't get home until I'm 56
They say I'm the Messiah, you rappers is liars
I sign up at church just to rap at the choir
To spit my new verse at y

In [29]:
spotify =spotipy.Spotify(auth=token)
spotify_id = []
for index, row in playlist_df.iterrows():
    try:
        res = spotify.search(row["Name"], type="track", limit=1)
        spotify_id.append(res["tracks"]["items"][0]["id"])
    except:
        text = row["Name"]
        head, sep, tail = text.partition('(')
        res = spotify.search(head, type="track", limit=1)
        spotify_id.append(res["tracks"]["items"][0]["id"])
        continue
        
playlist_df["Spotify ID"] = spotify_id
playlist_df.head()

,Name,Album,Artist,Lyrics,Spotify ID
0,Waves,The Life Of Pablo,Kanye West,[Intro]\nTurn it up!\n\n[Verse 1: Kanye West]\...,3nAq2hCr1oWsIU54tS98pL
1,Lord Knows (feat. Tory Lanez),Dreams Worth More Than Money,Meek Mill,[Chorus: Tory Lanez]\nAll I wanted was a new M...,57hCSu4zMTdnSum7NBL1Ye
2,GOOBA,GOOBA,6ix9ine,"[Chorus]\nI tell a nigga don't dick ride, don'...",4NhDYoQTYCdWHTvlbGVgwo
3,Moon Relate,Eternal Atake (Deluxe) - LUV vs. The World 2,Lil Uzi Vert,"[Intro]\nPain, I done got numb from the pain (...",4sWGHlZGM1XA7TyWdtBAbA
4,On God,JESUS IS KING,Kanye West,"[Intro]\nYo, Pi'erre, yo, Pi'erre\n\n[Verse]\n...",572BChNcXOIvtfn2TGmTKc


In [30]:
danceability = []
energy = []
key = []
loudness = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
duration_ms = []

for index, row in playlist_df.iterrows():
    try:
        track_features = spotify.audio_features(tracks=row['Spotify ID'])
        danceability.append(track_features[0]["danceability"])
        energy.append(track_features[0]["energy"])
        key.append(track_features[0]["key"])
        loudness.append(track_features[0]["loudness"])
        speechiness.append(track_features[0]["speechiness"])
        acousticness.append(track_features[0]["acousticness"])
        instrumentalness.append(track_features[0]["instrumentalness"])
        liveness.append(track_features[0]["liveness"])
        valence.append(track_features[0]["valence"])
        tempo.append(track_features[0]["tempo"])
        duration_ms.append(track_features[0]["duration_ms"])
    except:
        print(f"Couldn't find details for {row['Spotify ID']}")
        continue

In [31]:
playlist_df["danceability"] = danceability
playlist_df["energy"] = energy
playlist_df["key"] = key
playlist_df["loudness"] = loudness
playlist_df["speechiness"] = speechiness
playlist_df["acousticness"] = acousticness
playlist_df["instrumentalness"] = instrumentalness
playlist_df["liveness"] = liveness
playlist_df["valence"] = valence
playlist_df["tempo"] = tempo
playlist_df["duration_ms"] = duration_ms

In [32]:
playlist_df.head()

,Name,Album,Artist,Lyrics,Spotify ID,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,Waves,The Life Of Pablo,Kanye West,[Intro]\nTurn it up!\n\n[Verse 1: Kanye West]\...,3nAq2hCr1oWsIU54tS98pL,0.557,0.586,10,-4.570,0.0382,0.0528,0.000007,0.339,0.0681,96.091,181573
1,Lord Knows (feat. Tory Lanez),Dreams Worth More Than Money,Meek Mill,[Chorus: Tory Lanez]\nAll I wanted was a new M...,57hCSu4zMTdnSum7NBL1Ye,0.434,0.695,10,-7.464,0.4850,0.0168,0.000000,0.834,0.1600,167.309,318840
2,GOOBA,GOOBA,6ix9ine,"[Chorus]\nI tell a nigga don't dick ride, don'...",4NhDYoQTYCdWHTvlbGVgwo,0.611,0.688,1,-5.688,0.3410,0.0264,0.000000,0.251,0.3930,178.462,132303
3,Moon Relate,Eternal Atake (Deluxe) - LUV vs. The World 2,Lil Uzi Vert,"[Intro]\nPain, I done got numb from the pain (...",4sWGHlZGM1XA7TyWdtBAbA,0.867,0.768,7,-3.730,0.0791,0.1800,0.000000,0.254,0.3400,102.021,181826
4,On God,JESUS IS KING,Kanye West,"[Intro]\nYo, Pi'erre, yo, Pi'erre\n\n[Verse]\n...",572BChNcXOIvtfn2TGmTKc,0.797,0.722,5,-6.230,0.0601,0.0925,0.000000,0.112,0.3380,145.990,250272
